Usaremos una función del módulo calib3d, es decir, cv.findHomography() . Si pasamos el conjunto de puntos de ambas imágenes, encontrará la transformación de perspectiva de ese objeto. Luego podemos usar cv.perspectiveTransform() para encontrar el objeto. Se necesitan al menos cuatro puntos correctos para encontrar la transformación.

Hemos visto que puede haber algunos errores posibles durante la comparación que pueden afectar el resultado. Para resolver este problema, el algoritmo utiliza RANSAC o LEAST_MEDIAN (que pueden decidirse mediante las banderas). Por lo tanto, las buenas coincidencias que proporcionan una estimación correcta se denominan valores internos y las restantes se denominan valores atípicos. cv.findHomography() devuelve una máscara que especifica los puntos internos y externos.

In [ ]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
MIN_MATCH_COUNT = 10
img1 = cv.imread('box.png', cv.IMREAD_GRAYSCALE)          # queryImage
img2 = cv.imread('box_in_scene.png', cv.IMREAD_GRAYSCALE) # trainImage
# Initiate SIFT detector
sift = cv.SIFT_create()
# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(img1,None)
kp2, des2 = sift.detectAndCompute(img2,None)
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks = 50)
flann = cv.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(des1,des2,k=2)
# store all the good matches as per Lowe's ratio test.
good = []
for m,n in matches:
    if m.distance < 0.7*n.distance:
        good.append(m)

Ahora establecemos la condición de que haya al menos 10 coincidencias (definidas por MIN_MATCH_COUNT) para encontrar el objeto. De lo contrario, simplemente muestre un mensaje que indique que no hay suficientes coincidencias.

Si se encuentran suficientes coincidencias, extraemos las ubicaciones de los puntos clave coincidentes en ambas imágenes. Se pasan para encontrar la transformación de perspectiva. Una vez que obtenemos esta matriz de transformación de 3x3, la usamos para transformar las esquinas de queryImage en los puntos correspondientes en trainImage

In [ ]:
if len(good)>MIN_MATCH_COUNT:
    src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
    dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
    M, mask = cv.findHomography(src_pts, dst_pts, cv.RANSAC,5.0)
    matchesMask = mask.ravel().tolist()
    h,w = img1.shape
    pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
    dst = cv.perspectiveTransform(pts,M)
    img2 = cv.polylines(img2,[np.int32(dst)],True,255,3, cv.LINE_AA)
else:
    print( "Not enough matches are found - {}/{}".format(len(good), MIN_MATCH_COUNT) )
    matchesMask = None

Finalmente dibujamos nuestros valores internos (si encontramos el objeto con éxito) o puntos clave coincidentes (si fallamos).



In [ ]:
draw_params = dict(matchColor = (0,255,0), # draw matches in green color
                   singlePointColor = None,
                   matchesMask = matchesMask, # draw only inliers
                   flags = 2)
img3 = cv.drawMatches(img1,kp1,img2,kp2,good,None,**draw_params)
plt.imshow(img3, 'gray'),plt.show()